# INTRO
so this notebook is to clean up the data based on the decisions made in ML_eda.ipynb, without all the graphs and text. I will probably convert this into a .py file.

## Load in the fact table from bq
is this going to work outside of bigquery studio? Might need a dockerfile and poetry

In [28]:
import pandas as pd
import numpy as np
import pandas_gbq
from google.cloud import bigquery
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [12]:
client = bigquery.Client(project="dt-grad-emea3-cap-dev")
query = """SELECT * FROM `dt-grad-emea3-cap-dev.dataform.fct_vehicle_listings`"""
result = client.query(query)

df = pandas_gbq.read_gbq(query, project_id='dt-grad-emea3-cap-dev')

/Users/charles.doyne/Code/emea3-capstone-ml/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [13]:
df = df[['price', 'model', 'fuel', 'odometer', 'title_status', 'transmission', 'cylinder_count', 'car_year', 'car_condition', 'state']]

Do we need to do error handling.
E.g.
 - Are there negative prices? Shouldn't be, think it's impossible because craigslist wouldn't allow. If there are just remove them and carry on
 - Are the environment variables valid inputs?

Things that could be environment variables


In [ ]:
PRICE_LOWER_BOUND = 1000
PRICE_UPPER_BOUND = df['price'].quantile(0.999) # this has to be changed to a percentile or a fixed value to be configurable by the client

TOP_X_MODELS = 30

ODO_LOWER_BOUND = 1000
ODO_UPPER_BOUND = 500000

CAR_YEAR_THRESHOLD = 2000

In [15]:
# only keeping rows with prices between our chosen price range
bounded_price_df =  df[(df['price'] >= PRICE_LOWER_BOUND) & (df['price'] <= PRICE_UPPER_BOUND)]

In [16]:
# Only including rows that are from the top X (30 atm) models (they will have enough data)
# we could change this to set a threshold limit for the min amount of data a model has e.g. 400 datapoints per model to be included
top_models = bounded_price_df['model'].value_counts().nlargest(TOP_X_MODELS).index
top_models_df = bounded_price_df[bounded_price_df['model'].isin(top_models)]

In [17]:
bounded_odometer_df = top_models_df[(top_models_df['odometer'] >= ODO_LOWER_BOUND) & (top_models_df['odometer'] <= ODO_UPPER_BOUND)]

In [18]:
# only including rows that have 4,6,8 cylinders as there is little data for the other options
cylinder_filtered_df = bounded_odometer_df[bounded_odometer_df['cylinder_count'].isin([4, 6, 8])]


In [19]:
# only including cars that are from years since the CAR_YEAR_THRESHOLD
filtered_by_year_df = cylinder_filtered_df[cylinder_filtered_df['car_year'] >= CAR_YEAR_THRESHOLD]

In [20]:
# only keeping rows with cars that have good or excellent condition
filtered_condition_df = filtered_by_year_df[filtered_by_year_df['car_condition'].isin(['good', 'excellent'])]

In [21]:
# only keeping automatic transmission
transmission_df = filtered_condition_df[filtered_condition_df['transmission'] == 'automatic']

In [22]:
# only keeping clean cars
title_status_df = transmission_df[transmission_df['title_status']== 'clean']

In [23]:
# only keeping gas cars
fuel_df = title_status_df[title_status_df['fuel'] == 'gas']

In [25]:
# dropping leftover unwanted columns
car_df = fuel_df.drop(columns=['fuel', 'title_status', 'transmission'])

Normalising and encoding for ML

In [29]:
car_df['car_condition'] = car_df['car_condition'].map({'good': 0, 'excellent': 1})
scaler = StandardScaler()
car_df['odometer'] = scaler.fit_transform(car_df[['odometer']])
car_df = pd.get_dummies(car_df, columns=['model'])
car_df = pd.get_dummies(car_df, columns=['state'])

Fixing it up for bigquery

In [31]:
car_df.columns = car_df.columns.str.replace('-', '_')
car_df.columns = car_df.columns.str.replace(' ', '')

In [32]:
train_df, test_df = train_test_split(car_df, test_size=0.15, random_state=42)

tbf we might not need to send it back to bigquery. We could just save it locally and train an ML model on it in python using scikit or smth.

In [ ]:
train_df.to_gbq(destination_table='Machine_learning.training_car_data', project_id='dt-grad-emea3-cap-dev', if_exists='replace')
test_df.to_gbq(destination_table='Machine_learning.test_car_data', project_id='dt-grad-emea3-cap-dev', if_exists='replace')